# Performing gradient on remote tensors

In [88]:
import torch as th

In [89]:
import syft as sy

In [90]:
anum = sy.VirtualWorker(hook,id="anum")

In [91]:
x = th.tensor([1.,2,3,4],requires_grad=True).send(anum)
y = th.tensor([1.],requires_grad=True).send(anum)

In [92]:
x

(Wrapper)>[PointerTensor | me:19504699817 -> anum:97251796553]

In [93]:
y

(Wrapper)>[PointerTensor | me:30193878936 -> anum:78250466618]

In [94]:
z = (x+y).sum()

In [95]:
z.backward()

(Wrapper)>[PointerTensor | me:13839466283 -> anum:86551994779]

In [96]:
x = x.grad
print(x)

(Wrapper)>[PointerTensor | me:19504699817 -> anum:97251796553]::grad


In [97]:
x = x.clone().get()

In [98]:
x

tensor([1., 1., 1., 1.])

# Build a simple linear Model

In [99]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True).send(anum)
target = th.tensor([[1.],[1],[0],[0]],requires_grad=True).send(anum)
weights = th.tensor([[0.],[0]],requires_grad=True).send(anum)

In [100]:
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad *= 0
    print(loss.get().data)
    

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


(Wrapper)>[PointerTensor | me:83872147555 -> anum:23197075887]

(Wrapper)>[PointerTensor | me:97370772003 -> anum:90380030616]::data